In [1]:
import ee
ee.Authenticate()
ee.Initialize()
import geemap
geemap.ee_initialize()

In [2]:
#generuje mape
Map = geemap.Map(center=[54.52, 19.16], zoom=10)
#definiuje poligon
region = ee.Geometry.BBox(18.58, 54.69, 19.79, 54.34)
#konwertuje region na FeatureColecction
fc = ee.FeatureCollection(region)
#definiuje styl do regionu
style = {'color': 'ffff00ff', 'fillColor': '00000000'}
#dodaje region do mapy 
Map.addLayer(fc.style(**style), {}, 'poligon')
#pobieram dane o pokryciu terenu dla calego swiata
landcover = ee.ImageCollection('ESA/WorldCover/v200')
#przycinam dane landcover do regionu
landcover_region_clipped = landcover.map(lambda image: image.clip(region))
#pobieram collection danych sentinel, one nie są potrzebne ale przykład
collection = (
    ee.ImageCollection('COPERNICUS/S2_SR') #slaba jakosc
    .filterDate('2020-07-06', '2020-08-09')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 15)) # tu robie to samo ale z filtrami
)
#wyciagam mediane
image = collection.median()
#przyciolem se
image = image.clip(region) #przyciolem se
vis = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
}
#dodaje land_cover do mapy
Map.addLayer(landcover_region_clipped, {}, 'landcover')
#dodaje dane sentinela do mapy 
Map.addLayer(image, vis, 'Sentinel-2')

Map

Map(center=[54.52, 19.16], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [10]:
landcover.getInfo() #Fajna funkcja do informacji landcover

{'type': 'ImageCollection',
 'bands': [],
 'version': 1699537813333727,
 'id': 'ESA/WorldCover/v200',
 'properties': {'max_mirrored_version': 1699489608680902},
 'features': [{'type': 'Image',
   'bands': [{'id': 'Map',
     'data_type': {'type': 'PixelType',
      'precision': 'int',
      'min': 0,
      'max': 255},
     'dimensions': [4320000, 1728000],
     'crs': 'EPSG:4326',
     'crs_transform': [8.333333333333333e-05,
      0,
      -180,
      0,
      -8.333333333333333e-05,
      84]}],
   'version': 1699537784392512,
   'id': 'ESA/WorldCover/v200/2021',
   'properties': {'Map_class_names': ['Tree cover',
     'Shrubland',
     'Grassland',
     'Cropland',
     'Built-up',
     'Bare / sparse vegetation',
     'Snow and ice',
     'Permanent water bodies',
     'Herbaceous wetland',
     'Mangroves',
     'Moss and lichen'],
    'system:time_start': 1609455600000,
    'system:footprint': {'type': 'LinearRing',
     'coordinates': [[-180, -90],
      [180, -90],
      [180,

In [11]:
# Pobranie pierwszego obrazu z kolekcji
first_image = landcover_region_clipped.first()

# Utworzenie maski dla klas, które nie są równe 80
land_mask = first_image.select('Map').neq(80)

# Przefiltrowanie obrazu na podstawie maski
land_layer = first_image.updateMask(land_mask)

# Dodanie warstwy wody do mapy
Map.addLayer(land_layer, {'palette': ['blue']}, "Land")

# Wyświetlenie mapy
Map

Map(center=[54.52, 19.16], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [12]:

# Definicja funkcji dylatacji
def dilate(image, radius):
    dilated = image.focal_max(radius, 'circle', 'meters')
    return dilated

# Pobranie pierwszego obrazu z kolekcji
first_image = landcover_region_clipped.first()

# Utworzenie maski dla klas, które nie są równe 80
land_mask = first_image.select('Map').neq(80)

# Przefiltrowanie obrazu na podstawie maski
land_layer = first_image.updateMask(land_mask)

# Dylatacja warstwy wody
dilated_land_layer = dilate(land_layer, 100)  # Promień dylatacji - możesz dostosować według potrzeb

# Dodanie warstwy lądu po dylatacji do mapy
Map.addLayer(dilated_land_layer, {'palette': ['green']}, "Dilated Water Bodies")

# Wyświetlenie mapy
Map

Map(center=[54.52, 19.16], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [13]:
# Konwersja obrazu lądów po dylatacji na binarny obraz
land_binary = dilated_land_layer.gt(0)

In [14]:
# Utworzenie maski dla całego obszaru
full_extent_mask = ee.Image.constant(1).clip(region)
# Odjęcie obszaru wody od całego obszaru
# Zastąpienie wartości None zerami w masce wody
land_binary = land_binary.unmask(0)
#Map.addLayer(land_binary, {'palette': ['white', 'blue']}, 'unmask')


# Odwrócenie maski lądowej
water_mask = land_binary.Not()

# Dodanie warstwy maski wody do mapy
Map.addLayer(water_mask, {'palette': ['white', 'blue']}, 'Water Mask')


clipped_image = image
# Zastosowanie tej maski do obrazu Sentinel-2
image_without_water = clipped_image.updateMask(water_mask)

# Dodanie warstwy obrazu Sentinel-2 bez obszaru wody do mapy #Tu jest warstwa do obróbki 
Map.addLayer(image_without_water, vis, 'full_extent_mask')

# Wyświetlenie mapy
Map

Map(center=[54.52, 19.16], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [15]:
#Utworzenie kolekcji obrazów Sentinel-1 GRD
collection = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(region)
    .filterDate('2020-07-06', '2020-08-09')
)

images = collection.limit(10)  # Wybierz pierwszych 10 zobrazowań (możesz dostosować tę liczbę)

# Przycięcie każdego zobrazowania do określonego regionu
clipped_images = images.map(lambda img: img.clip(region))

# Połączenie zobrazowań w jeden obraz
mosaic_image = clipped_images.mosaic()

# Ustawienie wizualizacji
vis_params = {'min': -25, 'max': 0}

# Dodanie obrazu do mapy
Map.addLayer(mosaic_image, vis_params, 'Sentinel-1 VV (Mosaic)')

#wycięcie morza
radar_mosaic_sea = mosaic_image.updateMask(water_mask)
Map.addLayer(water_mask, {'palette': ['white', 'blue']}, 'Water Mask')
Map.addLayer(radar_mosaic_sea, vis_params, 'Sentinel-1 VV (moze)')
# Wyświetlenie mapy
Map

Map(center=[54.52, 19.16], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [16]:
#binaryzuje statki
statki = radar_mosaic_sea.select('VV').gt(2)

# Dylatacja warstwy wody
#dilated_land_layer = dilate(land_layer, 100)  # Promień dylatacji - możesz dostosować według potrzeb

# Dodanie warstwy lądu po dylatacji do mapy
# Ustawienie pikseli z wartością VV = 0 na None
statki = statki.updateMask(statki.neq(0))
dilated_statki_layer = dilate(statki, 500)
# Dodaj warstwę statków do mapy z przezroczystością
Map.addLayer(dilated_statki_layer, {'palette': ['yellow']}, "Statki")



# Wyświetl mapę
Map

Map(center=[54.52, 19.16], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…